# 2.1 神经元（下）

在上一节中，我们用回归模型实现了一个简单的函数拟合，同时介绍了一些神经网络的基本概念。在本节，我们将探索神经网络更具有泛化的模型编写，同时实现更多基于pytorch这一由Meta（Facebook母公司）开发的深度学习框架的底层实现（这个框架的所有算法都由c/c++编写，再跟python代码绑定），我们将重点实现这一框架独有的动态图，这一有别于其余深度学习框架（Tensorflow）的静态图的独特实现。

## 数字
首先我们实现一个数字类，这个类有一个更独特的学名：张量（Tensor）

In [1]:
class Tensor:
    def __init__(self, data):
        self.data = data

    def __add__(self, other):
        out = Tensor(self.data + other.data)
        return out

    def __mul__(self, other):
        out = Tensor(self.data * other.data)
        return out
    
    def __repr__(self):
        return f"Tensor(data={self.data})"

我们定义了一个名叫Tensor的类。实现了针对加法和乘法的运算符重载，同时我们还实现了魔术方法\_\_repr\_\_用于打印我们的结果，我们来测试一下这个类：

In [2]:
a = Tensor(12)
b = Tensor(15)
print(a + b)
print(a * b)

Tensor(data=27)
Tensor(data=180)


回顾上一节我们代码实现反向传播的过程，我们都是首先计算一个损失函数，获得以可训练参数作为自变量的函数，然后对每个可训练参数计算偏导数，最后再计算用这个偏导数乘上步长，用来计算这次迭代后可训练参数的结果

同时，根据链式法则，我们并不需要一口气就算出某个可训练参数的偏导数，我们可以用递归的方式逐层求偏导，算出这个可训练参数的偏导数，现在让我们修改这段代码，先算出这些基本运算符的导数，用于后续的递归

In [2]:
class Tensor:
    def __init__(self, data):
        self.data = data
        self.grad = 0.0
        self._backward = lambda: None

    def __add__(self, other):
        out = Tensor(self.data + other.data)
        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward
        return out

    def __mul__(self, other):
        out = Tensor(self.data * other.data)
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        return out
    
    def __repr__(self):
        return f"Tensor(data={self.data})"

(ps: grad是累加的，因为一个Tensor可能被用于计算多个Tensor的值)

让我们来测试一个一个函数f(x)=a*b+c，当a=10,b=5,c=10时，分别计算f(x)对a,b,c的偏导，可以很容易的算出(∂f/∂a) = 5,(∂f/∂b) = 10, (∂f/∂c) = 1

In [3]:
a = Tensor(10)
b = Tensor(5)
c = Tensor(10)
d = a*b
e = d+c
e.grad=1
e._backward()
print(d.grad)
print(c.grad)
d._backward()
print(a.grad)
print(b.grad)

1.0
1.0
5.0
10.0


现在，我们需要实现反向传播算法，我们需要保证算式计算偏导数的先后顺序，我们必须保证在链式法则中，算式中后算的值的偏导数被先算出来。其实熟悉算法的同学应该能知道我们需要用什么算法实现这一功能了，那就是拓扑排序。

## 拓扑排序
在计算机科学领域，有向图的拓扑排序（英语：Topological sorting）或拓扑测序（英语：Topological ordering）是对其顶点的一种线性排序，使得对于从顶点u 到顶点v的每个有向边uv，u 在排序中都在v之前。我们来实现Tensor类的反向传播算法，同时因为我们的反向传播是在整个算式算到最外面开始的，为了确保反向传播的顺序，我们需要在等式计算的时候，记录下通过计算得到当前值的tensor序列，我们用_prev表示

In [4]:
class Tensor:
    def __init__(self, data, _prev=()):
        self.data = data
        self.grad = 0.0
        self._backward = lambda: None
        self._prev = set(_prev)

    def __add__(self, other):
        out = Tensor(self.data + other.data, (self, other))
        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward
        return out

    def __mul__(self, other):
        out = Tensor(self.data * other.data, (self, other))
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        return out
    
    def backward(self):
    
        topo = []
        # 记录下已访问过的Tensor集合
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)
        # 因为上面得到的topo序列顺序是反的，所以我们在逐步计算偏导数时需要先进行逆序操作
        self.grad = 1.0
        for node in reversed(topo):
            node._backward()
    
    def __repr__(self):
        return f"Tensor(data={self.data})"

让我们再测试一遍函数f(x)=a*b+c，当a=10,b=5,c=10时，分别计算f(x)对a,b,c的偏导，可以很容易的算出(∂f/∂a) = 5,(∂f/∂b) = 10, (∂f/∂c) = 1

In [6]:
a = Tensor(10)
b = Tensor(5)
c = Tensor(10)
e = a*b+c
e.backward()
print(a.grad)
print(b.grad)
print(c.grad)

5.0
10.0
1.0


现在，我们大体上已经实现了一个Tensor类，我们现在还需要补上激活函数，来实现我们上一节的遗留问题

In [ ]:
import math
class Tensor:
    def __init__(self, data, _prev=()):
        self.data = data
        self.grad = 0.0
        self._backward = lambda: None
        self._prev = set(_prev)

    def __add__(self, other):
        out = Tensor(self.data + other.data, (self, other))
        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward
        return out

    def __mul__(self, other):
        out = Tensor(self.data * other.data, (self, other))
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        return out
    
    def tanh(self):
        x = self.data
        t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
        out = Tensor(t, (self, ), 'tanh')
        
        def _backward():
            self.grad += (1 - t**2) * out.grad
        out._backward = _backward
        
        return out
    
    def sigmoid(self):
        x = self.data
        s = 1 / (1 + math.exp(-x))
        out = Tensor(s, (self,), 'sigmoid')

        def _backward():
            self.grad += (s * (1 - s)) * out.grad
        out._backward = _backward

        return out

    def relu(self):
        x = self.data
        r = max(0, x)
        out = Tensor(r, (self,), 'relu')

        def _backward():
            self.grad += (1 if x > 0 else 0) * out.grad
        out._backward = _backward

        return out

    def backward(self):
        topo = []
        # 记录下已访问过的Tensor集合
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)
        # 因为上面得到的topo序列顺序是反的，所以我们在逐步计算偏导数时需要先进行逆序操作
        self.grad = 1.0
        for node in reversed(topo):
            node._backward()
    
    def __repr__(self):
        return f"Tensor(data={self.data})"